In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
df.drop(columns=['total_revenue'], inplace=True)

In [ ]:
df['date'] =  pd.to_datetime(df['date'], infer_datetime_format=True)

In [ ]:
df['imi'] = df['measurable_impressions'].apply(lambda x: weird_division(1, x))

In [ ]:
df.describe().T

In [ ]:
train = df[df['date'] <= "2019-06-21"]
test = df[df['date'] > "2019-06-21"]

test = test[test['CPM'] >= 0]
high = test['CPM'].quantile(0.95)
test = test[test['CPM'] < high]

train = train[train['CPM'] < high]

train = shuffle(train)
test = shuffle(test)

In [ ]:
train.shape, test.shape

In [ ]:
X_train = train.drop(columns=['CPM', 'date'])
y_train = train['CPM']
X_test = test.drop(columns=['CPM', 'date'])
y_test = test['CPM']

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

In [ ]:
model = AdaBoostRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
model = CatBoostRegressor()
model.fit(X_train, y_train, logging_level='Silent')
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)